In [7]:
#| hide
from chasqui import *

# Chasqui

> A lightweight workflow automation system for computational materials science

`chasqui` helps you manage VASP calculations on remote HPC clusters with PBS queue systems. It handles job submission, monitors queue status, and tracks results—all while respecting strict authentication requirements and queue limits.

**Key features:**
- Local SQLite database tracks all jobs
- Batched SSH operations (2FA-friendly)
- Self-perpetuating remote queue (no cron needed)
- Automatic overflow management (respects PBS limits)
- Simple CLI for everyday tasks

### The Need for Simple Workflow Systems

Running computational materials calculations on HPC clusters often involves:
- Submitting hundreds of jobs with varying parameters
- Monitoring queue status across multiple runs
- Managing data transfer between local and remote systems
- Respecting queue policies (maximum jobs, walltime limits)
- Dealing with authentication barriers (2FA, SSH keys)

Existing workflow systems like AiiDA, FireWorks, or Snakemake are powerful but can be heavy—requiring databases, web servers, or complex configuration. For individual researchers or small groups, there's a gap: **we need something lightweight, transparent, and easy to understand.**

`chasqui` fills this gap by embracing simplicity:

- **Single SQLite file** - no database server
- **Plain bash scripts** - no daemons or background processes
- **SSH and PBS only** - standard HPC tools 
- **Literate programming** - readable notebooks, not opaque frameworks

### Why "Chasqui"?

The name comes from the **chasqui** (also *chaski*), the relay messengers of the Inca Empire. These runners formed a sophisticated communication network spanning thousands of kilometers across the Andes, carrying messages and small goods between administrative centers. Chasquis often carried **khipus** (*quipus*)—intricate systems of knotted strings that encoded numerical data, records, and possibly narratives. 

### Design Philosophy

1. **Transparency** — You should understand what's happening  
2. **Simplicity** — Fewer moving parts means less to break  
3. **Flexibility** — Work with your HPC environment, not against it  
4. **Literate** — Code as documentation, documentation as code  

`chasqui` isn't trying to be the most feature-rich workflow system. It's trying to be the one you can **debug at 2 AM when your calculations are due**, understand six months later when you return to a project, and modify when your HPC center changes queue policies.

## Usage

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/{{user}}/{{lib_name}}.git
```

or from [conda][conda]

```sh
$ conda install -c {{user}} {{lib_path}}
```

or from [pypi][pypi]


```sh
$ pip install {{lib_path}}
```


[repo]: {{git_url}}
[docs]: https://{{user}}.github.io/{{lib_name}}/
[pypi]: https://pypi.org/project/{{lib_name}}/
[conda]: https://anaconda.org/{{user}}/{{lib_name}}

## How to use

Fill me in please! Don't forget code examples:

In [8]:
1+1+1

3

### Documentation

Documentation can be found hosted on this GitHub [repository][repo]'s [pages][docs]. Additionally you can find package manager specific guidelines on [conda][conda] and [pypi][pypi] respectively.

[repo]: https://github.com/GarciaJC/chasqui
[docs]: https://garciajc.github.io/chasqui/
[pypi]: https://pypi.org/project/{{lib_name}}/
[conda]: https://anaconda.org/{{user}}/{{lib_name}}